In [1]:
import chess
import chess.pgn
import glob
import numpy as np

def board_to_tensor(board: chess.Board) -> np.ndarray:
    tensor = np.zeros((12, 8, 8), dtype=np.uint8)

    piece_to_index = {
        (chess.PAWN, chess.WHITE): 0,
        (chess.KNIGHT, chess.WHITE): 1,
        (chess.BISHOP, chess.WHITE): 2,
        (chess.ROOK, chess.WHITE): 3,
        (chess.QUEEN, chess.WHITE): 4,
        (chess.KING, chess.WHITE): 5,
        (chess.PAWN, chess.BLACK): 6,
        (chess.KNIGHT, chess.BLACK): 7,
        (chess.BISHOP, chess.BLACK): 8,
        (chess.ROOK, chess.BLACK): 9,
        (chess.QUEEN, chess.BLACK): 10,
        (chess.KING, chess.BLACK): 11,
    }

    for square, piece in board.piece_map().items():
        row = 7 - chess.square_rank(square) # Flip for correct orientation
        col = chess.square_file(square)
        idx = piece_to_index[(piece.piece_type, piece.color)]
        tensor[idx, row, col] = 1

    return tensor

def move_to_tensor(move: chess.Move) -> np.ndarray:
    start_mask = np.zeros((8, 8), dtype=np.uint8)
    end_mask = np.zeros((8, 8), dtype=np.uint8)

    start_row = 7 - chess.square_rank(move.from_square)
    start_col = chess.square_file(move.from_square)

    end_row = 7 - chess.square_rank(move.to_square)
    end_col = chess.square_file(move.to_square)

    start_mask[start_row, start_col] = 1
    end_mask[end_row, end_col] = 1
    return np.stack([start_mask, end_mask])

In [2]:
from stockfish import Stockfish
stockfish = Stockfish(path="./stockfish-ubuntu-x86-64-avx2")

path = 'data/5_less.pgn'

num_games = 0
endplay_games = 0

inputs = []
outputs = []

for filepath in glob.glob(path):
    with open(filepath, encoding="utf-8", errors="ignore") as file:
        while(True):
            game = chess.pgn.read_game(file)
            if game is None:
                break
            num_games += 1
            if (num_games % 500) == 0:
                print(num_games)
            final_board = game.end().board()
            # display(final_board)
            num_pieces = len(final_board.piece_map())
            if (game.headers["Result"] == "1-0"): # just look at white wins for now, as convention
                end_node = game.end()
                node = end_node
                for _ in range(5): # go back 5 plies
                    node = node.parent
                if node.board().turn == chess.WHITE:
                    node = node.parent
                # display(node.board())
                stockfish.set_fen_position(node.board().fen())
                evaluation = stockfish.get_evaluation()
                # print(stockfish.get_evaluation())
                if ((evaluation['type'] == 'cp') and (evaluation['value']) > 300) or ((evaluation['type'] == "mate") and (evaluation['value'] > 0)): # white is winning
                    x = board_to_tensor(node.board())
                    y = move_to_tensor(node.move)
                    inputs.append(x)
                    outputs.append(y)
                    endplay_games += 1

print(num_games, endplay_games)

500


no matching legal move for e2e4 (e2 -> e4) in 6k1/4P3/r3R3/4K3/8/8/8/8 b - - 10 64 while parsing <Game at 0x767267a66d70 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'f1' in 8/8/8/8/5b2/8/4Kpk1/5N2 w - - 3 77 while parsing <Game at 0x767267ab8b20 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'ga1' in 8/8/1P6/3k3p/8/4B3/7K/8 w - - 5 63 while parsing <Game at 0x76725eb026b0 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'g1' in 8/8/8/p5P1/1Rk4K/8/8/8 b - - 0 62 while parsing <Game at 0x767267ab8fa0 ('?' vs. '?', '????.??.??' at '?')>
no matching legal move for e2e4 (e2 -> e4) in 8/3K4/5P1p/7k/8/3B4/8/8 b - - 0 66 while parsing <Game at 0x76725f5a02e0 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'h2' in 8/7K/4k3/6P1/6r1/8/8/8 w - - 0 85 while parsing <Game at 0x76725eb4e020 ('?' vs. '?', '????.??.??' at '?')>


1000


no matching legal move for e2e4 (e2 -> e4) in 8/K2k4/1P6/8/8/8/8/8 b - - 2 60 while parsing <Game at 0x76725eb4da20 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'g1' in 8/6p1/8/5k1p/7P/6K1/8/8 w - - 0 57 while parsing <Game at 0x76725ff32680 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'g1' in 8/2k5/4K3/1pP5/4B3/8/8/8 b - - 0 60 while parsing <Game at 0x76725f593370 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'ch5' in 8/8/8/8/8/5pk1/6N1/1r4K1 w - - 4 53 while parsing <Game at 0x76725ffbd030 ('?' vs. '?', '????.??.??' at '?')>


1500
2000


...

12000


illegal san: 'a1' in 8/8/8/n2B4/5k2/2K5/6P1/8 b - - 5 59 while parsing <Game at 0x76725ffbc040 ('?' vs. '?', '????.??.??' at '?')>
no matching legal move for e2e4 (e2 -> e4) in 5k2/5N1P/6K1/8/8/8/8/8 b - - 0 113 while parsing <Game at 0x76725f51be80 ('?' vs. '?', '????.??.??' at '?')>


12500


illegal san: 'ha1' in 8/2K5/P3R3/6k1/8/8/r7/8 b - - 6 68 while parsing <Game at 0x76725ff47ca0 ('?' vs. '?', '????.??.??' at '?')>


12904 6095


In [3]:
X = np.stack(inputs)
Y = np.stack(outputs)

np.savez_compressed("data/5_less_tensor.npz", inputs=X, outputs=Y)